<a href="https://colab.research.google.com/github/Madhav-Somanath/SBSPS-Challenge-693-Predicting-The-Energy-Output-of-Wind-Turbine-Based-on-Weather-Condition/blob/master/Data_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
from datetime import datetime, date, timedelta
import time
import types
import pandas as pd

df = pd.read_csv('./Data/scada_kaggle.csv')
df

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286
...,...,...,...,...,...
50525,31 12 2018 23:10,2963.980957,11.404030,3397.190793,80.502724
50526,31 12 2018 23:20,1684.353027,7.332648,1173.055771,84.062599
50527,31 12 2018 23:30,2201.106934,8.435358,1788.284755,84.742500
50528,31 12 2018 23:40,2515.694092,9.421366,2418.382503,84.297913


In [4]:
df.shape

(50530, 5)

In [5]:
world_weather_keys = ["151d22887cd746c8b30170119200305","a2a4c031e1174540953170617200305","3b8d0ddf95874ebc963171315200305","a7a8a673626349d6b2b171426200305",
                      "2c7acfdf3cd94dd6980171732200305","f2d087202dd74d3b8e9171937200305","dc721b5c73524d18bdc171944200305","a26775efe1e249b0a07172328200305",
                      "08ba377d157442f7b02172339200305","b32f31b4b4434128b88172618200305","54c5c4326ada4f189fc172631200305","c6f169f91683422f8ba172832200305",
                      "a39f242f81784b21808173201200305","e16a33c0b36040069aa173413200305","a39f242f81784b21808173201200305","01efbb6048bb48cc894173827200305",
                      "539ecc5092544eb087f173940200305","3d1b0704663e46e18f4174348200305","a339dad955e74b37a53174426200305","e2d4e57fb25f4535abf174810200305",
                      "e2fca3c1ea1a41c395f174907200305","f381fc760b034b70805175107200305","87ceb9281638401ca07175304200305","2c5dab9b79eb4daa9fb175507200305",
                      "e486c0fcdba54022b52150217200505","73160037869d421c95f150902200505","9c61f783fcd9467fb8e151028200505","25ac72b90a114c04963151054200505",
                      "40b9b6abd1ee4f3bab6151141200505","871cfe6fa20f440399f151257200505","36b3fc4cd1bc41aa9d7155722200505",
                      "f07f174181434b5e9e3155957200505","f280bed306334e0f9a2160217200505",
                      "6d401b23e9164970bc9160321200505","32309a1471524556b73160532200505","6af18246b3c748488f0160649200505",
                      "bd0cc1e7a4c14783bb1160745200505","65b031b57a9c41b5825161035200505","03ed5a8e69f5440087a161038200505","b17a228605634f84b51161207200505",
                      "f0b2df13be40426da7e161307200505","74b95c5a3847420d860161401200505","221fcd82d40f412fa77161510200505","d904fcc09e794699933161642200505",
                      "ca5283987239404aa31161717200505","cb7c0e83d493492081e161810200505","3b66bdc3b2ea4d2dabb161835200505","16f6c54894074e1bbdd162020200505",
                      "b6c7e4787cab459e801162135200505","a0d9bc141f2048dba26162253200505","fa5e3c18fbee4effbc0162451200505","4ee182c9c70842a088c162813200505",
                      "6dc633552a294225b6f162953200505","d7fbe569cc264b20b83163138200505","4b28dd6161354edeacc163136200505","72ae499526354e70a19163246200505",
                      "15a1c3bb8a654226868115922200805","f8a4c7c3bdce47d586a120157200805","f407d13de31b41458fb120349200805","f407d13de31b41458fb120349200805",
                      "12b837a2b8274581999120708200805","e5020deb05254b42a18120958200805","d9025e89120a45bfa14121205200805","323e7b9519e54cfda52121929200805",
                      "d5ba31b28f1148d4ab0122129200805","94a8bafb43684c80997122528200805","c1cf9b909f394cfbae5180339200305","e0e7d07dd7084f8ab8c180347200305",
                      "c333cdb241b946af831180610200305","7e1a5b3b20d94ddc8cf180643200305","8379e7f1e660499099b180957200305","06586f0cb7c34d71bb3181258200305",
                      "bc2d266e519246aa8ee181358200305","2e847be35de14622b3c181655200305","a744e85c62ee4349969181704200305", "e4b388776ff341b792b72402201604",
                      "f524287934a14cd982272606201604", "3f55fc3f3bd64eb0bdd72753201604", "1ff77be29feb4ea38f672944201604", "8f325a6d818c4bd0af473135201604", 
                      "e3a954bff9704341ad573438201604", "b5cb7e93f8d84929b1773939201604", "2200fef7e22b4b7a88c74303201604", "fe786af6899b48239c275234201604",
                      "72179825bc344a4293a75539201604", "638eccbd256b4366a4405417201004","cab55e4edc5746b1b2b05810201004","8ddbd8055b364aec90e10120201004",
                      "0fdc1b0c6eb54ec3a3f10457201004","3bd0ac243ddd44cf91010645201004","c81847db4b1c4b71aef10907201004",
                      "11b9ba9546d547cca4111312201004","7d8905af73ae44dfa2111522201004","0ff2b8f5d610495aa5711737201004",
                      "3f9ee0b1dc5d4037a1712155201004","3707325f27a54e5a86c12413201004","0fbeaab1b25544d6a5612823201004",
                      "0c7bd5cdde6b47f8a0150332202004","6858063b07824b7386b50547202004","e0867652b6b3466ebac50757202004",
                      "d867251c213449f48b851238202004","25a0182d23814ddc81b51446202004","82f02cd7ecb0447c8f651638202004",
                      "e2f9bfb059944a0490051825202004","e12e6c1abccf47e7ab152151202004","adbbbaec5cf046c8a4652350202004",
                      "5ca8918327084a2ebbf52541202004","7de5edee38ba456280852831202004","c19b5a0f767b4b61b0253019202004",
                      "02aca4e57cd442159ac53244202004","9bab12b72c8f4b6cb5d53559202004","c14f3cb92df24551bef53816202004",
                      "88c4d0a7060244b988054015202004","f1928f4162a94ab09fa54226202004","0081cb08de7f45b687d54521202004",
                      "ec927b918e9c436f8f0132045202004","5fbadaa705e54af8a45132313202004","4fe092d6213b4754995132444202004",
                      "0b840a8a671c453eb48132642202004","66e9cd20248b4735ad6132835202004","f1c27eaf9a3b49858b6133321202004",
                      "2cedc140babd429488d133523202004","b0cfda025e4743daa43133703202004","076b7de9241b47a5b8f133835202004",
                      "dd282c191ea647b0bfd134043202004","482ba2e356594f048b7134234202004","1654a84ea17048f4995134418202004",
                      "acc60e0fdbab42daa83134613202004","5eb463dfdc5d436ba88134816202004","34def415e62e44de976135004202004",
                      "07d2ac83254044fc92a135155202004","455d1b95e8474272964135343202004","f939c3f727eb46478a0135529202004",
                      "4b6a700637554250bad140000202004","760d2952fdb2440b9b6140246202004","febfd6293d25460591c75731202104",
                      "7361ac700d7c40ebb2675910202104","d384d3c0b88842b088483502202104","a72ea582edff4fb9a9d90545202104",
                      "954356550d2d4e31ab990731202104","12402144412b44a3bb290848202104","3494faf7c7304393bf391011202104"
                     ]

In [6]:
len(world_weather_keys)

142

In [16]:
from wwo_hist import retrieve_hist_data

In [18]:
from urllib.error import HTTPError
frequency=1

final_data_new = []

count = 0

current_key_count = 0

key_track = 0
current_key = world_weather_keys[key_track]

max_key_count = 500

requests_time_out = 0

lat_long = ["40.6549,29.2842"]
for i in df.values:
    end_date = datetime.strptime(i[0], '%d %m %Y %H:%M')
    start_date = end_date
    one_row = []

    if current_key_count > max_key_count:
        key_track += 1
        current_key = world_weather_keys[key_track]
        current_key_count = 0
    try:
        hist_weather_data = retrieve_hist_data(current_key,
                                  lat_long,
                                  start_date,
                                  end_date,
                                  frequency,
                                  location_label = False,
                                  export_csv = False,
                                  store_df = True
                                  )
        current_key_count += 1
        
        records = hist_weather_data[0]
        records = records[["maxtempC", "WindGustKmph", "DewPointC", "WindChillC", "humidity", "precipMM", "pressure"]].apply(pd.to_numeric)
        
        maxtempC = records["maxtempC"].values
        windGustKmph = records["WindGustKmph"].values
        DewPointC = records["DewPointC"].values
        windChillC = records["WindChillC"].values
        humidity = records["humidity"].values
        precipMM = records["precipMM"].values
        pressure = records["pressure"].values         
        
        one_row.append([ end_date,  sum(maxtempC) / len(maxtempC), sum(windGustKmph) / len(windGustKmph), 
                         sum(DewPointC) / len(DewPointC), sum(windChillC) / len(windChillC),
                         sum(humidity) / len(humidity), sum(precipMM) / len(precipMM), sum(pressure) / len(pressure)])

    except HTTPError:
        requests_time_out += 1

    final_data_new.extend(one_row)  

    count += 1

    if count % 1000 == 0:
        print("Count:{}, Len(Final data):{}, key track:{}".format(count, len(final_data_new), key_track))




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-01 to 2018-01-01




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-01-02 to 2018-01-02


...



Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-14 to 2018-02-14




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-15 to 2018-02-15




Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


Retrieving weather data for 40.6549,29.2842


Currently retrieving data for 40.6549,29.2842: from 2018-02-16 to 2018-02-16


URLError: <urlopen error timed out>

In [ ]:
df = pd.DataFrame( data_integrated, columns = ["DateTime", "LV ActivePower", "Wind Speed (m/s)", "Theoretical_Power_Curve (KWh)", "Wind Direction",
                                         "end_date",  "maxtempC", "windGustKmph", "DewPointC", "windChillC", "humidity", "precipMM", "pressure"] )

In [ ]:
df.drop(df.columns[0], axis=1, inplace = True)

In [ ]:
df["end_date"] = df["end_date"].apply(pd.to_datetime)

In [ ]:
lVActivePower = df["LV ActivePower"]
df.drop("LV ActivePower", axis = 1, inplace = True)

In [ ]:
df["LV ActivePower"] = lVActivePower
df.head()

In [ ]:
from project_lib import Project
project = Project(project_id='<ProjectID>', project_access_token='<ProjectAccessToken>')
pc = project.project_context

In [ ]:
from project_lib import Project
project = Project(pc,'<ProjectID>','<ProjectAccessToken>')

project.save_data(file_name = "integrated_data.csv",data = df.to_csv(index=False))